In [1]:
from prettytable import PrettyTable
from project.models import Plan

qs = Plan.objects.all()
qs = qs.select_related("project")
qs = qs.order_by("project__name", "id")

# show available plan to let you choose an id
x = PrettyTable(field_names=["id", "Nom du projet", "Nom du plan"], align="l")
x.add_rows([(p.id, p.project.name, p.name) for p in qs])
print(x)

+----+------------------+----------------+
| id | Nom du projet    | Nom du plan    |
+----+------------------+----------------+
| 4  | COBAS_SHAPE      | Waza           |
| 2  | Vallée de l'Eyre | Plan numéro 2  |
| 3  | Vallée de l'Eyre | Hello the plan |
+----+------------------+----------------+


In [2]:
plan = Plan.objects.get(pk=4)
plan

<Plan: Waza>

In [5]:
from tempfile import TemporaryDirectory
from pathlib import Path
from zipfile import ZipFile

temp_dir_path = Path(TemporaryDirectory().name)
temp_dir_path

PosixPath('/tmp/tmpo3bw50qk')

In [7]:
file_stream = plan.shape_file.open()
with ZipFile(file_stream) as zip_file:
        zip_file.extractall(temp_dir_path)  # extract files to dir
files_path = [_ for _ in temp_dir_path.iterdir() if _.suffix == ".shp"]
files_path

[PosixPath('/tmp/tmpo3bw50qk/emprise_arrondissement_Arcachon.shp')]

In [8]:
from django.contrib.gis.gdal import DataSource
ds = DataSource(files_path[0])
print(ds)

/tmp/tmpo3bw50qk/emprise_arrondissement_Arcachon.shp (ESRI Shapefile)


In [13]:
print(f"Nombre de layers: {len(ds)}")
layer = ds[0]
print(f"Premier layer: {layer}")
print(f"Nombre d'items dans ce layer: {len(layer)}")
print(f"Type de champ géométrique: {layer.geom_type}")

Nombre de layers: 1
Premier layer: emprise_arrondissement_Arcachon
Nombre d'items dans ce layer: 1
Type de champ géométrique: Polygon


In [14]:
# fields disponibles
layer.fields

['Id']

In [16]:
# expected fields
from project.models import PlanEmprise

PlanEmprise.mapping

{'mpoly': 'MULTIPOLYGON',
 'name': 'NAME',
 'description': 'DESCRIPTION',
 'lot': 'LOT',
 'surface': 'SURFACE',
 'us_code': 'US_CODE',
 'cs_code': 'CS_CODE'}

In [21]:
# what should be in mapping
# rq: mpoly require to be there
set(layer.fields).intersection(set(PlanEmprise.mapping.values())).union(set(['MULTIPOLYGON']))

{'MULTIPOLYGON'}